importing libraries

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras import models,preprocessing
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.layers import Input,LSTM,Dense,Embedding
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle


In [2]:
#initialize all variables 
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()


In [3]:
#read dataset file
with open('try.txt','r',encoding='utf-8') as f:
    rows=f.read().split('\n')
  

In [4]:
for row in rows[0:4]:
    #split input and target by '\t'=tab
    input_text,target_text = row.split('\t')
    #add '\t' at start and '\n' at end of text.
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text.lower())
    target_texts.append(target_text.lower())
    #split character from text and add in respective sets
    input_characters.update(list(input_text.lower()))
    target_characters.update(list(target_text.lower()))
   
   
    


In [5]:
input_texts

['go.', "i'm 19.", "i'm ok.", 'i love you.']

In [6]:
target_characters

{'\t',
 '\n',
 ' ',
 '!',
 '.',
 '1',
 '9',
 'ं',
 'आ',
 'क',
 'ग',
 'च',
 'ज',
 'त',
 'ब',
 'म',
 'र',
 'व',
 'स',
 'ह',
 'ा',
 'ु',
 'े',
 'ो'}

In [7]:
#sort input and target characters 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [8]:
#get the total length of input and target characters
num_en_chars = len(input_characters)
num_dec_chars = len(target_characters)

In [9]:
#get the maximum length of input and target text.
max_input_length = max([len(i) for i in input_texts])
max_target_length = max([len(i) for i in target_texts])



In [11]:
def bagofcharacters(input_texts,target_texts):
  #inintialize encoder , decoder input and target data.
  en_in_data=[] ; dec_in_data=[] ; dec_tr_data=[]
  #padding variable with first character as 1 as rest all 0.
  pad_en=[1]+[0]*(len(input_characters)-1)
  pad_dec=[0]*(len(target_characters)) ; pad_dec[2]=1
  #countvectorizer for one hot encoding as we want to tokenize character so
  #anlyzer is true and None the stopwords action.
  cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char')
  for i,(input_t,target_t) in enumerate(zip(input_texts,target_texts)):
    #fit the input characters into the CountVectorizer function
    cv_inp= cv.fit(input_characters)
    
    #transform the input text from the help of CountVectorizer fit.
    #it character present than put 1 and 0 otherwise.
    en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
    cv_tar= cv.fit(target_characters)		
    dec_in_data.append(cv_tar.transform(list(target_t)).toarray().tolist())
    #decoder target will be one timestep ahead because it will not consider 
    #the first character i.e. '\t'.
    dec_tr_data.append(cv_tar.transform(list(target_t)[1:]).toarray().tolist())
    
    #add padding variable if the length of the input or target text is smaller
    #than their respective maximum input or target length. 
    if len(input_t) < max_input_length:
      for _ in range(max_input_length-len(input_t)):
        en_in_data[i].append(pad_en)
    if len(target_t) < max_target_length:
      for _ in range(max_target_length-len(target_t)):
        dec_in_data[i].append(pad_dec)
    if (len(target_t)-1) < max_target_length:
      for _ in range(max_target_length-len(target_t)+1):
        dec_tr_data[i].append(pad_dec)
  
  #convert list to numpy array with data type float32
  en_in_data=np.array(en_in_data,dtype="float32")
  dec_in_data=np.array(dec_in_data,dtype="float32")
  dec_tr_data=np.array(dec_tr_data,dtype="float32")
  en_in_data

  return en_in_data,dec_in_data,dec_tr_data

In [12]:
#create input object of total number of encoder characters
en_inputs = Input(shape=(None, num_en_chars))

In [13]:
#create LSTM with the hidden dimension of 256
#return state=True as we don't want output sequence.
encoder = LSTM(256, return_state=True,return_sequences=True)

In [14]:
#discard encoder output and store hidden and cell state.
en_outputs, state_h, state_c = encoder(en_inputs)
en_states = [state_h, state_c]

In [15]:
#create input object of total number of decoder characters
dec_inputs = Input(shape=(None, num_dec_chars))

In [16]:
decoder_embedding = Embedding( num_dec_chars, 256 , mask_zero=True) (dec_inputs)

In [17]:
#create LSTM with the hidden dimension of 256
#return state and return sequences as we want output sequence.
dec_lstm = LSTM(256, return_sequences=True, return_state=True)

In [18]:
#initialize the decoder model with the states on encoder.
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=en_states)

In [19]:
#Output layer with shape of total number of decoder characters 
dec_dense = Dense(num_dec_chars, activation="softmax")
dec_outputs = dec_dense(dec_outputs)

In [20]:
#create Model and store all variables 
model = Model([en_inputs, dec_inputs], dec_outputs)
pickle.dump({'input_characters':input_characters,'target_characters':target_characters,
             'max_input_length':max_input_length,'max_target_length':max_target_length,
             'num_en_chars':num_en_chars,'num_dec_chars':num_dec_chars},open("training_data.pkl","wb"))


In [21]:
num_en_chars

15

In [22]:
#load the data and train the model
en_in_data,dec_in_data,dec_tr_data = bagofcharacters(input_texts,target_texts)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [en_in_data, dec_in_data],
    dec_tr_data,
    batch_size=64,
    epochs=300,
    validation_split=0.0,
)
# Save model
model.save("s2s")

#summary and model plot
model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:551: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


Epoch 1/300
1/1 [==============================] - 9s 9s/step - loss: 3.1833 - accuracy: 0.0326
Epoch 2/300
1/1 [==============================] - 0s 145ms/step - loss: 3.1087 - accuracy: 0.4674
Epoch 3/300
1/1 [==============================] - 0s 136ms/step - loss: 3.0290 - accuracy: 0.5435
Epoch 4/300
1/1 [==============================] - 0s 141ms/step - loss: 2.9349 - accuracy: 0.5435
Epoch 5/300
1/1 [==============================] - 0s 134ms/step - loss: 2.8096 - accuracy: 0.5326
Epoch 6/300
1/1 [==============================] - 0s 131ms/step - loss: 2.6156 - accuracy: 0.5326
Epoch 7/300
1/1 [==============================] - 0s 138ms/step - loss: 2.2919 - accuracy: 0.5326
Epoch 8/300
1/1 [==============================] - 0s 133ms/step - loss: 1.9552 - accuracy: 0.5326
Epoch 9/300
1/1 [==============================] - 0s 141ms/step - loss: 2.1659 - accuracy: 0.5326
Epoch 10/300
1/1 [==============================] - 0s 125ms/step - loss: 2.2032 - accuracy: 0.5326
Epoch 11/300

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 15)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 24)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  278528      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  287744      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [27]:
 def decode_sequence(input_seq):
        datafile = pickle.load(open("training_data.pkl","rb"))
        input_characters = datafile['input_characters']
        target_characters = datafile['target_characters']
        max_input_length = datafile['max_input_length']
        max_target_length = datafile['max_target_length']
        num_en_chars = datafile['num_en_chars']
        num_dec_chars = datafile['num_dec_chars']
      
        #Inference model
        #load the model
        model = models.load_model("s2s")
        #construct encoder model from the output of second layer
        #discard the encoder output and store only states.
        enc_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
        #add input object and state from the layer.
        en_model = Model(model.input[0], [state_h_enc, state_c_enc])

        #create Input object for hidden and cell state for decoder
        #shape of layer with hidden or latent dimension
        dec_state_input_h = Input(shape=(256,), name="input_3")
        dec_state_input_c = Input(shape=(256,), name="input_5")
        dec_states_inputs = [dec_state_input_h, dec_state_input_c]

        #add input from the encoder output and initialize with 
        #states.
        dec_lstm = model.layers[3]
        dec_outputs, state_h_dec, state_c_dec = dec_lstm(
            model.input[1], initial_state=dec_states_inputs
        )
        dec_states = [state_h_dec, state_c_dec]
        dec_dense = model.layers[4]
        dec_outputs = dec_dense(dec_outputs)
        #create Model with the input of decoder state input and encoder input
        #and decoder output with the decoder states.
        dec_model = Model(
            [model.input[1]] + dec_states_inputs, [dec_outputs] + dec_states
        )
        #create dict object to get character from the index.
        reverse_target_char_index = dict(enumerate(target_characters))
        #get the states from the user input sequence
        states_value = en_model.predict(input_seq)

        #fit target characters and 
        #initialize every first character to be 1 which is '\t'.
        #Generate empty target sequence of length 1.
        cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char') 
       
        co=cv.fit(target_characters) 
        target_seq=np.array([co.transform(list("\t")).toarray().tolist()],dtype="float32")
        
        #if the iteration reaches the end of text than it will be stop the it
        stop_condition = False
        #append every predicted character in decoded sentence
        decoded_sentence = ""
        while not stop_condition:
            #get predicted output and discard hidden and cell state.
            output_chars, h, c = dec_model.predict([target_seq] + states_value)
            
            #get the index and from dictionary get character from it.
            char_index = np.argmax(output_chars[0, -1, :])
            text_char = reverse_target_char_index[char_index]
            decoded_sentence += text_char

            # Exit condition: either hit max length
            # or find stop character.
            if text_char == "\n" or len(decoded_sentence) > max_target_length:
                stop_condition = True
            #update target sequence to the current character index.
            target_seq = np.zeros((1, 1, num_dec_chars))
            target_seq[0, 0, char_index] = 1.0
            states_value = [h, c]
        #return the decoded sentence
        return decoded_sentence


In [28]:
def bagofcharacters(input_t):
        datafile = pickle.load(open("training_data.pkl","rb"))
        input_characters = datafile['input_characters']
        target_characters = datafile['target_characters']
        max_input_length = datafile['max_input_length']
        max_target_length = datafile['max_target_length']
        num_en_chars = datafile['num_en_chars']
        num_dec_chars = datafile['num_dec_chars']
        cv=CountVectorizer(binary=True,tokenizer=lambda txt: txt.split(),stop_words=None,analyzer='char') 
        en_in_data=[] ; pad_en=[1]+[0]*(len(input_characters)-1)

        cv_inp= cv.fit(input_characters)
        en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())

        if len(input_t)< max_input_length:
          for _ in range(max_input_length-len(input_t)):
            en_in_data[0].append(pad_en)
    
        return np.array(en_in_data,dtype="float32")

In [29]:
def entry():
    x=  input( 'Enter eng sentence : ' ) 
    en_in_data = bagofcharacters(x.lower()+".")
    x=decode_sequence(en_in_data)
    print(x)

In [30]:
 
entry()

Enter eng sentence :  i love you


 हांव तुजेर मोग करता.

